# Model Acquisition into IncQuery Server

## Set up

In [1]:
import iqs_jupyter
from iqs_jupyter import schema
iqs = iqs_jupyter.connect()

## Optional: clean up

In [3]:
iqs.persistent_index.delete_all_persisted_model_compartments()
iqs.in_memory_index.delete_all_inmemory_model_compartments()
iqs.queries.unregister_all_queries()

{'message': 'complete'}

## Identify model by compartment URI, check URI is vacant, start acquisition

In [4]:
model = schema.ModelCompartment(compartment_uri = "foo:/bar/baz")

In [5]:
model not in iqs.persistent_index.list_persisted_model_compartments().persisted_model_compartments

True

In [6]:
start_response = iqs.acquisition.start_acquisition(model)
start_response

{'compartment_uri': 'foo:/bar/baz',
 'write_handle': 'ecb35d3e-8b01-4ddc-946e-895a0aac7953'}

## Upload model contents chunk by chunk

In [7]:
chunk_data = {
      "elementStore": [
        {
          "classifier": {
            "classifierName": "Class",
            "packageNsUri": "http://www.nomagic.com/magicdraw/UML/2.5.1"
          },
          "elements": [
            {
              "compartmentURI": "foo:/bar/baz",
              "relativeElementID": "class_Foo"
            },
            {
              "compartmentURI": "foo:/bar/baz",
              "relativeElementID": "class_Bar"
            }
          ]
        }
      ],
      "attributeStore": [
        {
          "attribute": {
            "name": "name",
            "ownerProxy": {
              "classifierName": "NamedElement",
              "packageNsUri": "http://www.nomagic.com/magicdraw/UML/2.5.1"
            },
              "typeProxy" : {
                "classifierName" : "String",
                "packageNsUri" : "http://www.nomagic.com/magicdraw/UML/2.5.1"
              }
          },
          "tuples": [
            {
              "source": {
                "compartmentURI": "foo:/bar/baz",
                "relativeElementID": "class_Foo"
              },
              "values": [
                "Class Foo"
              ]
            },
            {
              "source": {
                "compartmentURI": "foo:/bar/baz",
                "relativeElementID": "class_Bar"
              },
              "values": [
                "Class Bar"
              ]
            }
          ]
        }
      ],
      "referenceStore": [
        {
          "reference": {
              "name" : "redefinedElement",
              "ownerProxy" : {
                "classifierName" : "RedefinableElement",
                "packageNsUri" : "http://www.nomagic.com/magicdraw/UML/2.5.1"
              },
              "typeProxy" : {
                "classifierName" : "RedefinableElement",
                "packageNsUri" : "http://www.nomagic.com/magicdraw/UML/2.5.1"
              },
              "oppositeProxy" : {
                "featureName" : "_redefinableElementOfRedefinedElement",
                "featureKind" : "EREFERENCE",
                "classifierProxy" : {
                  "classifierName" : "RedefinableElement",
                  "packageNsUri" : "http://www.nomagic.com/magicdraw/UML/2.5.1"
                }
              }
          },
          "tuples": [
            {
              "source": {
                "compartmentURI": "foo:/bar/baz",
                "relativeElementID": "class_Bar"
              },
              "targets": [
                {
                  "compartmentURI": "foo:/bar/baz",
                  "relativeElementID": "class_Foo"
                }
              ]
            }
          ]
        }
      ],
      "singleCompartment": False
    }

In [9]:
chunk_request = schema.UploadChunkRequest(
  data_chunk = schema.PersistIndexData(
    write_handle = start_response.write_handle,
    compartment_uri = start_response.compartment_uri,
    index_data = chunk_data,
    all_updates_from_primary_model = True
  )
)
upload_response = iqs.acquisition.upload_acquisition_chunk(chunk_request)
upload_response

{'message': 'Chuck received'}

## Do not forget to close & save model when done (or discard if aborted)

In [10]:
iqs.acquisition.close_acquisition(start_response)

{'write_handle': 'ecb35d3e-8b01-4ddc-946e-895a0aac7953'}

## Use newly acquired model on server

In [11]:
model in iqs.persistent_index.list_persisted_model_compartments().persisted_model_compartments

True

In [12]:
iqs.in_memory_index.load_model_compartment(model)

{'message': 'completed',
 'statistics': {'attribute_store_size': 1,
                'attribute_store_tuples': 2,
                'direct_instances_store_size': 2,
                'element_store_size': 1,
                'element_store_tuples': 2,
                'merged_load_cost': 2,
                'merged_unique_elements': 2,
                'reference_store_size': 1,
                'reference_store_tuples': 1}}

In [13]:
test_query_package = "test.queries"
test_query_main = "test.queries.redefines"
test_query_code = '''

pattern redefines(
    src: Class, 
    srcName: java String, 
    trg: Class, 
    trgName: java String
) {
    RedefinableElement.redefinedElement(src, trg);
    Class.name(src, srcName);
    Class.name(trg, trgName);
}
'''

In [14]:
if test_query_main not in iqs.queries.list_queries().viatra.query_fq_ns: # skip if already registered
    try: 
        iqs.queries.register_queries_plain_text(test_query_code, query_package=test_query_package)
    except: 
        print("Query registration not available as guest user; please try again in a few minutes")
else:
    print("Query is already registered; proceed")

In [15]:
iqs.query_execution.execute_query_on_model_compartment(
    schema.ExecuteQueryOnCompartmentRequest(model_compartment = model, query_fqn = test_query_main)
)

,src,srcName,trg,trgName
,,,,
0,element #class_Bar,Class Bar,element #class_Foo,Class Foo
